In [1]:
# Workflow Agents in Google ADK: Itinerary-Driven Packing List Example
# Welcome 👋🏻 In this notebook, you'll learn how to use Workflow Agents in Google's Agent Development Kit (ADK)
# to chain together two LLM agents: one that creates a travel itinerary, and another that generates a packing list
# based on that itinerary. This mirrors how real travelers plan: first decide what you'll do, then pack accordingly!

In [2]:
# load API Key set in .EnvironmentError
import dotenv
dotenv.load_dotenv()

import os
#import getpass

# Load the Gemini API key from environment variable
# If not set, prompt the user to enter it
api_key = os.environ.get("GEMINI_API_KEY")
if not api_key:
    print("GEMINI_API_KEY not found in environment variables.")
    #api_key = getpass.getpass("Enter your GEMINI_API_KEY: ")
    #os.environ["GEMINI_API_KEY"] = api_key

print("API key configured successfully." if api_key else "Failed to set API key.")

API key configured successfully.


In [3]:
# Install Google ADK for Python
# This is the foundational package that provides all the necessary components for building and running your agents.
# The --quiet flag suppresses verbose output during installation.
%pip install google-adk --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Verify ADK Installation (Optional but Recommended)
%pip show google-adk

Name: google-adk
Version: 1.18.0
Summary: Agent Development Kit
Home-page: https://google.github.io/adk-docs/
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: c:\Users\gabri\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: anyio, authlib, click, fastapi, google-api-python-client, google-cloud-aiplatform, google-cloud-bigtable, google-cloud-discoveryengine, google-cloud-secret-manager, google-cloud-spanner, google-cloud-speech, google-cloud-storage, google-genai, graphviz, mcp, opentelemetry-api, opentelemetry-exporter-gcp-logging, opentelemetry-exporter-gcp-monitoring, opentelemetry-exporter-gcp-trace, opentelemetry-exporter-otlp-proto-http, opentelemetry-resourcedetector-gcp, opentelemetry-sdk, pydantic, python-dateutil, python-dotenv, PyYAML, requests, sqlalchemy, sqlalchemy-spanner, starlette, tenacity, typing-extensions, tzlocal, uvicorn, watchdog, websockets
Required-by: 
Note: you may need to restart the kernel to use updat

In [5]:
# Configure environment
import os

# Set GOOGLE_GENAI_USE_VERTEXAI to "False" to use the public Gemini API directly,
# rather than routing through Google Cloud's Vertex AI. This simplifies setup for quick demos.
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Define the specific Gemini model we'll use. 
# 'gemini-2.0-flash' is a fast and efficient model.
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

In [6]:
# Import Necessary Modules

# Agent: The fundamental class for creating an AI agent within ADK.
from google.adk.agents import Agent

# SequentialAgent: A specialized agent that chains together multiple sub-agents in a predefined sequence.
from google.adk.agents import SequentialAgent

# Runner: The orchestrator that manages the flow of messages and agent execution.
from google.adk.runners import Runner

#  InMemorySessionService: A simple, in-memory service for managing conversation sessions. 
# Sessions are vital for maintaining context in multi-turn dialogues.
from google.adk.sessions import InMemorySessionService

# google_search: An ADK tool that provides agents with the ability to perform web searches, enabling them to retrieve up-to-date information.
from google.adk.tools import google_search

# types from google.genai: Provides data structures (like Content and Part) to represent messages exchanged with the language model.
from google.genai import types

# IPython.display: Used for rendering rich content like Markdown in Jupyter notebooks.
from IPython.display import Markdown, display

In [7]:
# Itinerary Agent: Definition
# This agent is designed to create a detailed travel itinerary for a given destination and duration.
# It leverages the `google_search` tool to find up-to-date recommendations for attractions, restaurants, and activities.
# The `output_key="itinerary"` specifies that this agent's primary output will be accessible under the key "itinerary",
# which is crucial for passing this output to subsequent agents in a workflow.
itinerary_agent = Agent(
    name="itinerary_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="An agent that creates a travel itinerary for a given destination and trip duration, using Google Search for up-to-date recommendations.",
    instruction="""
        You are a helpful and creative travel itinerary planner.
        When the user provides a destination and trip duration, use the [google_search] tool to find current and popular attractions, restaurants, and activities for each day.
        For each day, create a detailed schedule with 2-4 activities, including at least one meal suggestion and one local attraction, prioritizing the user's interests (such as art and food).
        Present the itinerary in a clear, easy-to-read markdown format, organized by day.

        Example format:

        ### Day 1
        - Morning: [Attraction or activity]
        - Lunch: [Restaurant or food experience]
        - Afternoon: [Attraction or activity]
        - Evening: [Dinner suggestion or event]

        ### Day 2
        ...

        Always use the [google_search] tool to find the latest recommendations for each activity or restaurant.
        When providing recommendations, if a direct and relevant official website or information page URL is prominently found in the search results for a specific item, always include it as a Markdown hyperlink like [Item Name](URL). try to add as many hyperlinks as possible!
        Be concise, friendly, and ensure the itinerary is complete for all days requested.
    """,
    tools=[google_search], # This is the crucial part: assigning the tool to the agent!
    output_key="itinerary"  # This agent's output will be stored with the key "itinerary".
)

print("`itinerary_agent` successfully defined with `google_search` tool.")

`itinerary_agent` successfully defined with `google_search` tool.


In [8]:
# Packing List Agent: Definition
# This agent's primary role is to generate a comprehensive packing list.
# Crucially, its instruction includes a placeholder `{itinerary}`, which tells the agent to
# expect and use the output from a previous step (the itinerary_agent in this workflow).
# The `output_key="packing_list"` ensures that its final output is stored under this key.
packing_list_agent = Agent(
    name="packing_list_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Generates a packing list based on itinerary, destination, and user preferences.",
    instruction="""
        You are a helpful travel assistant.
        The user will provide their destination, trip duration, and a detailed itinerary (including planned activities, locations, and types of events for each day).
        Your task is to generate a complete, practical packing list tailored to the specific activities and locations in the itinerary.
        
        Itinerary: {itinerary} 

        **Thoroughly analyze the provided Itinerary, considering the specific activities, locations, and time of day for each entry.** Determine necessary clothing, gear, and items based *directly* on these plans. For example, if the itinerary mentions hiking, suggest hiking boots. If it mentions a fancy dinner, suggest formal wear.

        Organize the packing list by category (such as Clothing, Toiletries, Electronics, Documents, and Other Essentials).
        If you notice a special activity or requirement in the itinerary, include specific items for it and justify their inclusion based on the activity.
        If any information is missing, make reasonable assumptions based on the itinerary and destination—do not ask the user for more details unless absolutely necessary.
        
        **After the main categorized list, provide a dedicated 'Outfit Suggestions for Key Activities' section.** For 2-3 prominent activities from the itinerary, suggest a complete outfit including clothing, shoes, and any relevant accessories, explaining *why* it's suitable for that specific event.

        Present the complete output in a clear, easy-to-read markdown format. For example:

        #### Clothing
        - Comfortable walking shoes (for city tours and museum visits)
        - Lightweight jacket (for potentially cooler evenings in [Destination])
        - Dress for dinner at a nice restaurant (if formal dining is in itinerary)
        - Swimwear (if beach or pool is in itinerary)
        - ...

        #### Toiletries
        - Toothbrush and toothpaste
        - Sunscreen (for outdoor activities)
        - ...

        #### Electronics
        - Phone and charger
        - Plug adapter (if needed for [Destination])
        - ...

        #### Documents
        - Passport
        - Travel insurance (important for any trip abroad)
        - ...

        #### Other Essentials
        - Daypack for city tours
        - Reusable water bottle

        #### Outfit Suggestions for Key Activities
        *   **Day 1 - Morning (Tokyo National Museum):** Comfortable smart casual. E.g., Dark jeans/trousers, a layered top (blouse or stylish t-shirt), and comfortable, stylish walking shoes. A light scarf or jacket for cooler museum interiors.
        *   **Day 2 - Evening (Ninja Restaurant):** Fun, relaxed evening wear. E.g., A casual dress or nice shirt with dark pants. Comfortable shoes for easy movement.
        *   **Day 3 - Afternoon (Sushi Making Class):** Practical and comfortable. E.g., T-shirt, comfortable pants that allow movement, and closed-toe shoes. Avoid overly loose sleeves for hygiene and safety.

        Only ask clarifying questions if the itinerary is extremely vague or unclear. Otherwise, use your best judgment and generate a full, actionable packing list.
    """,
    output_key="packing_list"  # This agent's output will be stored with the key "packing_list".
)

In [9]:
# Sequential Agent: Definition
# The SequentialAgent allows you to define a workflow by chaining multiple sub-agents together.
# In this setup, the output of the first sub-agent (itinerary_agent) is automatically
# passed as input to the instruction of the next sub-agent (packing_list_agent)
# through the `{itinerary}` placeholder defined in the packing_list_agent's instruction.
wanderwise_agent = SequentialAgent(
    name="wanderwise_agent",
    sub_agents=[itinerary_agent, packing_list_agent],  # Defines the order of execution.
    description="Plans a travel itinerary and prepares a packing list based on activities planned."
)

In [10]:
# Import necessary modules for type hinting
from typing import Dict

# Modular Function for Agent Interaction (Corrected and Updated for multiple outputs)
async def run_adk_agent_interaction(
    agent: Agent,
    user_id: str,
    session_id: str,
    input_text: str,
    app_name: str = "default_app",
    session_service: InMemorySessionService = None,
) -> Dict[str, str]:
    """
    Runs an interaction with an ADK agent (can be a single Agent or SequentialAgent)
    and returns a dictionary of its final text responses.
    Each response is keyed by the agent's 'name' (event.agent_name),
    or a sequential 'response_N' if the agent_name is not available.

    Args:
        agent: The ADK Agent instance to interact with.
        user_id: A unique identifier for the user initiating the interaction.
        session_id: A unique identifier for the conversation session.
                    Using a new ID for each distinct example ensures isolation.
        input_text: The textual message from the user.
        app_name: The name of the application using the agent. Defaults to "default_app".
        session_service: An optional InMemorySessionService instance. If None, a new
                         instance is created, making each call self-contained.

    Returns:
        A dictionary where keys are the `agent_name` of the agent that produced the final response.
        Values are the full text responses.
        Returns an empty dictionary if no final response events are found.
    """
    # Use the provided session service if available, otherwise create a new one.
    if session_service is None:
        session_service = InMemorySessionService()

    # Create a new session.
    await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

    # Format the user's input into a Content object.
    content = types.Content(role="user", parts=[types.Part(text=input_text)])

    # Initialize the Runner.
    runner = Runner(agent=agent, app_name=app_name, session_service=session_service)

    # Run the agent.
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    # Collect all final responses
    all_final_responses = []
    for event in events:
        if event.is_final_response():
            # Concatenate all parts in the final response
            full_text = ''.join(part.text for part in event.content.parts if part.text)
            all_final_responses.append(full_text)
    return all_final_responses

print("Modular function `run_adk_agent_interaction` successfully updated to use `event.agent_name` for keys.")

Modular function `run_adk_agent_interaction` successfully updated to use `event.agent_name` for keys.


In [11]:
# Set up the session and user input
APP_NAME = "wanderwise_app"
USER_ID = "default_user"
SESSION_ID = f"{APP_NAME}_default_session"

#  Create a single instance of the session service.
# Passing a shared instance (`common_session_service`) to `run_adk_agent_interaction`
# is good practice, especially if you had a database-backed session service.
# For InMemorySessionService, it ensures all sessions are managed within the same memory scope.
common_session_service = InMemorySessionService()

# Define a list of diverse travel planning test cases for our workflow agent.
# Each dictionary contains a descriptive label, and the user's input.
test_cases = [
    {
        "label": "Adventure & Nature Trip (Banff)",
        "input_text": "I'm planning a 7-day trip to Banff National Park in August. I want to do moderate hiking and wildlife photography.",
    },
    {
        "label": "Culture & History Trip (Rome)",
        "input_text": "I'm a 35 yo male, and I am going to Rome for 5 days in October. I'm very interested in ancient history, classical art, and local cuisine.",
    },
    {
        "label": "Beach & Relaxation Trip (Maldives)",
        "input_text": "I'm visiting the Maldives for 4 days in January for relaxation and some water activities. 25F.",
    },
    {
        "label": "City Break & Shopping (Paris)",
        "input_text": "I'm doing a 3-day city break in Paris in April. I want to visit famous landmarks, do some shopping, and enjoy cafes. Love Emily in Paris!",
    }
]

print("------ [starting workflow agent interactions] ------\n")

# Iterate through each test case and run the travel planning workflow.
# The 'wanderwise_agent' is the SequentialAgent which orchestrates the 
# 'itinerary_agent' followed by the 'packing_list_agent'.
# The 'run_adk_agent_interaction' helper function handles session creation and
# collects all final responses (itinerary and packing list) into a list.
for i, case in enumerate(test_cases):
    print(f"\n--- Example {i+1}: {case['label']} ---")
    print(f"User Input: \"{case['input_text']}\"")

    # Invoke the workflow agent using the modular helper function.
    # We pass the shared 'common_session_service' instance.
    workflow_outputs_list = await run_adk_agent_interaction(
        agent=wanderwise_agent,
        user_id="user"+f"_{i+1}",
        session_id="session"+f"_{i+1}",  # Unique session ID for each test case
        input_text=case["input_text"],
        app_name=APP_NAME,
        session_service=common_session_service # Pass the shared service instance
    )

    # Parse and Display Outputs from the list.
    if workflow_outputs_list:
        for j, text_response in enumerate(workflow_outputs_list):
            display(Markdown(text_response))
    else:
        print(f"No outputs received for Example {i+1}.")
    
    print(f"--- Example {i+1}: Workflow Agent Interaction Complete ---")


print("\n------ [all workflow agent interactions complete] ------")


------ [starting workflow agent interactions] ------


--- Example 1: Adventure & Nature Trip (Banff) ---
User Input: "I'm planning a 7-day trip to Banff National Park in August. I want to do moderate hiking and wildlife photography."


Okay, I can help you plan a 7-day trip to Banff National Park in August with a focus on moderate hiking and wildlife photography. Here is a possible itinerary.

Here is a possible 7-day itinerary for Banff National Park, focusing on moderate hiking and wildlife photography:

### Day 1: Arrival in Banff and Exploring the Town

*   **Morning:** Arrive at Calgary International Airport (YYC) and drive to Banff (approximately 1.5 hours). Check into your hotel.
*   **Lunch:** Grab a quick bite at [Aardvark Pizza & Sub](), known for fast food.
*   **Afternoon:** Explore the town of Banff, visiting local shops and the [Banff Park Museum National Historic Site]().
*   **Evening:** Enjoy dinner at [The Maple Leaf](), offering Canadian cuisine with wild game and locally-raised meat.

### Day 2: Vermilion Lakes and Bow Valley Parkway

*   **Morning:** Head to Vermilion Lakes for sunrise photography, capturing reflections of Mount Rundle. This is a good spot for wildlife viewing, especially elk.
*   **Lunch:** Have lunch at the [Juniper Bistro](), which offers brunch and dinner.
*   **Afternoon:** Drive along the Bow Valley Parkway (Highway 1A), keeping an eye out for wildlife such as elk, deer, and coyotes. Stop at scenic viewpoints like [Castle Mountain]().
*   **Evening:** Dine at the [Waldhaus Restaurant](), known for its fondue experience and German beer.

### Day 3: Johnston Canyon and Ink Pots Hike

*   **Morning:** Hike the Johnston Canyon Trail to the Lower and Upper Falls.
*   **Lunch:** Enjoy a meal at the [Johnston Canyon Cafe & Market]().
*   **Afternoon:** Continue hiking to the Ink Pots, a series of cold mineral springs.
*   **Evening:** Relax and have dinner at [Banff Ave Brewing Co.](), a local brewery with a variety of beers and poutine.

### Day 4: Lake Louise and Plain of Six Glaciers

*   **Morning:** Drive to Lake Louise and take in the views of the turquoise lake and Victoria Glacier.
*   **Lunch:** Have lunch at the [Lake Agnes Tea House](), accessible by a moderate hike and offering light meals with views.
*   **Afternoon:** Hike the Plain of Six Glaciers Trail, a moderate hike with stunning mountain scenery and a tea house at the end.
*   **Evening:** Enjoy a fine dining experience at [Eden](), the Rimrock Resort’s restaurant.

### Day 5: Moraine Lake and Larch Valley

*   **Morning:** Visit Moraine Lake and hike the Rockpile Trail for panoramic views of the Valley of the Ten Peaks.
*   **Lunch:** Pack a picnic to enjoy by Moraine Lake.
*   **Afternoon:** Hike to Larch Valley and Sentinel Pass (if open), known for its stunning larch trees and views. Note: This area is popular, and access may be restricted; check Parks Canada for updates.
*   **Evening:** Have dinner at [Una Pizza + Wine](), known for its delicious Italian cuisine and wine.

### Day 6: Icefields Parkway and Bow Lake

*   **Morning:** Drive along the Icefields Parkway towards Jasper National Park, stopping at Bow Lake for photography.
*   **Lunch:** Stop at the [Whitehorn Bistro]() on Lake Louise Ski Resort for lunch with a view. Access it by taking the Lake Louise Summer Gondola.
*   **Afternoon:** Hike to Bow Glacier Falls, a 4.6 km trail with rewarding views.
*   **Evening:** Enjoy dinner at [Shoku Izakaya](), serving Asian food.

### Day 7: Sulphur Mountain and Departure

*   **Morning:** Hike up Sulphur Mountain or take the Banff Gondola to the summit for panoramic views of the Bow Valley.
*   **Lunch:** Have lunch at the summit of Sulphur Mountain.
*   **Afternoon:** Relax at the Banff Upper Hot Springs.
*   **Evening:** Depart from Calgary International Airport (YYC).

Enjoy your trip to Banff National Park!


Okay, here is a packing list tailored to your 7-day trip to Banff National Park in August, focusing on moderate hiking and wildlife photography, based on the itinerary provided.

#### Clothing

*   **Hiking Boots:** Sturdy, broken-in hiking boots are essential for the various trails.
*   **Hiking Socks:** Moisture-wicking hiking socks to prevent blisters. Several pairs.
*   **Moisture-Wicking Base Layers:** Tops and bottoms to regulate body temperature during hikes.
*   **Hiking Pants:** Quick-drying, convertible hiking pants are versatile.
*   **T-Shirts:** Several comfortable t-shirts for layering and casual wear.
*   **Long-Sleeved Shirts:** For sun protection and layering on cooler days.
*   **Fleece Jacket or Pullover:** For warmth during early mornings and evenings.
*   **Insulated Jacket:** A lightweight, packable insulated jacket for colder conditions, especially at higher elevations.
*   **Waterproof/Windproof Jacket:** Essential for unpredictable mountain weather.
*   **Rain Pants:** To stay dry during wet hikes.
*   **Casual Pants/Jeans:** For evenings in town and dining out.
*   **Nice Outfit:** For dinners at restaurants like The Maple Leaf and Eden.
*   **Underwear:** Adequate supply for the duration of the trip.
*   **Sleepwear:** Pajamas or comfortable sleep clothes.
*   **Swimsuit:** For Banff Upper Hot Springs on Day 7.
*   **Hat:** For sun protection.
*   **Gloves or Mittens:** Even in August, mornings and higher elevations can be chilly.
*   **Warm Scarf or Neck Gaiter:** For added warmth and protection from the elements.

#### Toiletries

*   **Sunscreen:** High SPF, essential for protecting skin at high altitude.
*   **Lip Balm with SPF:** To prevent chapped lips.
*   **Insect Repellent:** To protect against mosquitoes and other insects.
*   **Shampoo, Conditioner, and Soap:** Travel-sized bottles to save space.
*   **Toothbrush, Toothpaste, and Floss:**
*   **Deodorant:**
*   **Wet Wipes or Hand Sanitizer:** Useful for cleaning hands on the trail.
*   **Any Personal Medications:** With copies of prescriptions.
*   **Pain Relievers:** Such as ibuprofen or acetaminophen.
*   **First-Aid Kit:** Including blister treatment, antiseptic wipes, and bandages.

#### Electronics

*   **Camera:** With extra batteries and memory cards for wildlife photography.
*   **Camera Bag:** To protect your camera gear during hikes.
*   **Phone and Charger:**
*   **Portable Power Bank:** To charge devices on the go.
*   **Binoculars:** For wildlife viewing.
*   **Headlamp or Flashlight:** For early morning photography at Vermilion Lakes or in case of getting caught out on a trail after dark.
*   **Tripod:** For stable photography, especially in low light.

#### Documents

*   **Passport/ID:**
*   **Driver's License:**
*   **Travel Insurance Information:**
*   **Copies of Important Documents:** Keep separately from originals.
*   **Park Pass:** If required, purchase a Parks Canada Discovery Pass.
*   **Reservations:** Hotel, tour, and restaurant confirmations.

#### Other Essentials

*   **Daypack:** A comfortable backpack for carrying essentials on hikes.
*   **Reusable Water Bottles or Hydration Reservoir:** Staying hydrated is crucial.
*   **Water Filter or Purification Tablets:** For safe drinking water on longer hikes.
*   **Snacks:** Energy bars, trail mix, and other snacks for hikes.
*   **Bear Spray:** Essential for safety in bear country; know how to use it.
*   **Bear Bell:** To alert wildlife to your presence on trails.
*   **Hiking Poles:** For added stability on steep trails.
*   **Sunglasses:** With UV protection.
*   **Book or Entertainment:** For downtime in the evenings.
*   **Cash and Credit Cards:**
*   **Small Backpacking Towel:** Quick-drying and lightweight.
*   **Duct Tape:** For gear repairs.
*   **Compass/Map:** Although you'll likely have a phone, a physical map can be useful.
*   **Whistle:** For emergencies.
*   **Picnic Blanket:** For lunch at Moraine Lake.
*   **Dry bags:** To keep electronics and valuables safe from water damage.

#### Outfit Suggestions for Key Activities

*   **Day 3 - Johnston Canyon and Ink Pots Hike:**
    *   **Clothing:** Moisture-wicking t-shirt, hiking pants, fleece jacket, waterproof jacket (weather dependent).
    *   **Shoes:** Hiking boots with good ankle support.
    *   **Accessories:** Hat, sunglasses, small backpack with water, snacks, and bear spray.
    *   **Reasoning:** Layers are essential for changing conditions. Hiking boots provide traction and support on the trail.

*   **Day 5 - Moraine Lake and Larch Valley Hike:**
    *   **Clothing:** Moisture-wicking base layer, long-sleeved shirt, insulated jacket, hiking pants.
    *   **Shoes:** Hiking boots.
    *   **Accessories:** Hat, gloves, scarf, bear spray, water, and a picnic lunch.
    *   **Reasoning:** This hike is at a higher elevation, so warmer layers are necessary. Check trail conditions and pack accordingly.

*   **Day 4 - Dinner at Eden:**
    *   **Clothing:** Dress pants or skirt, blouse or button-down shirt, sweater or jacket.
    *   **Shoes:** Dress shoes or elegant boots.
    *   **Accessories:** Jewelry, scarf, or tie to elevate the outfit.
    *   **Reasoning:** Eden is a fine dining restaurant, so smart casual or semi-formal attire is appropriate.

Enjoy your trip to Banff National Park!


--- Example 1: Workflow Agent Interaction Complete ---

--- Example 2: Culture & History Trip (Rome) ---
User Input: "I'm a 35 yo male, and I am going to Rome for 5 days in October. I'm very interested in ancient history, classical art, and local cuisine."


Okay, I can help you plan a 5-day trip to Rome in October focusing on ancient history, classical art, and local cuisine. Here's a possible itinerary:

Okay, here is a detailed 5-day itinerary for your trip to Rome in October, focusing on ancient history, classical art, and local cuisine:

**Weather in Rome in October:** You can expect average daily temperatures ranging from 15°C (59°F) to 23°C (73°F). It's a good idea to pack layers, including a light jacket or sweater for the cooler mornings and evenings. There's a moderate chance of rain, so bring an umbrella or waterproof jacket.

### Day 1: Ancient Rome - Colosseum & Roman Forum

*   **Morning:** Start your journey at the **Colosseum**. Book a guided tour to explore the arena and learn about the gladiatorial contests and public spectacles that took place here. Consider the Colosseum Underground and Third Order Tour for a more in-depth experience.
*   **Lunch:** Enjoy traditional Roman pizza at **Forno Campo de' Fiori**.
*   **Afternoon:** Explore the **Roman Forum** and **Palatine Hill**. Wander through the ruins of ancient temples, government buildings, and public spaces, imagining life in ancient Rome. Palatine Hill offers panoramic views of the Forum and the city.
*   **Evening:** Have dinner at **La Taverna dei Fori Imperiali**, which serves Roman dishes with a twist near the Roman Forum.

### Day 2: Vatican City - Art & Religious History

*   **Morning:** Visit **Vatican City**. Explore the **Vatican Museums**, including the **Sistine Chapel** to admire Michelangelo's masterpiece. Book your tickets in advance to avoid long lines.
*   **Lunch:** Have lunch near Vatican City at **L'Archetto**, known for pizza.
*   **Afternoon:** Visit **St. Peter's Basilica**, one of the most important churches in Christendom. Climb to the top of the dome for panoramic views of Rome.
*   **Evening:** Enjoy an authentic Roman dinner at **Ristorante de Pecorino** in the Testaccio neighborhood.

### Day 3: Classical Art & Hidden Gems

*   **Morning:** Immerse yourself in classical art at the **Capitoline Museums**. See iconic Roman sculptures, including the equestrian statue of Marcus Aurelius and the Capitoline She-Wolf.
*   **Lunch:** Eat lunch at **Casa Manco** in Mercato Testaccio for pizza al taglio.
*   **Afternoon:** Discover the **Baths of Caracalla**, one of the largest thermal bath complexes of ancient Rome. Afterwards, explore the nearby **Appian Way**, an ancient Roman road lined with historic villas and monuments.
*   **Evening:** Experience Trastevere by having dinner at **Da Enzo al 29**, known for its authentic carbonara. Be prepared to wait, as they don't take reservations.

### Day 4: Ancient Wonders & Local Flavors

*   **Morning:** Explore the **Pantheon**, one of the best-preserved ancient Roman buildings.
*   **Lunch:** Have lunch near the Pantheon at **Salumeria Roscioli**, a deli-restaurant serving Roman classics.
*   **Afternoon:** Visit **National Roman Museum: Palazzo Massimo alle Terme**. See the stunning frescoes from Livia's villa and a collection of Roman and Greek sculptures.
*   **Evening:** Enjoy a wine tasting experience at **Il Goccetto** near Campo de' Fiori. For dinner, try **Trattoria Perilli** for traditional Roman dishes like carbonara.

### Day 5: Day Trip to Ostia Antica

*   **Morning:** Take a day trip to **Ostia Antica**, the ancient port of Rome. Explore the well-preserved ruins of this once-thriving city, including houses, shops, and public buildings.
*   **Lunch:** Have lunch at a restaurant in Ostia Antica.
*   **Afternoon:** Continue exploring Ostia Antica. Consider visiting the **Mithraeum of Circus Maximus**, an underground temple dedicated to the God Mithras.
*   **Evening:** Have a farewell dinner at **Armando al Pantheon**, known for traditional Roman cuisine.

**Additional Tips:**

*   **October Events:** Check out the **Rome Film Fest** if you are visiting between October 15th and 26th.
*   **Hidden Gems:** Consider visiting some hidden gems like the **Pyramid of Cestius**, the **Keyhole view of St. Peter's Basilica** or the **Baths of Diocletian**.
*   **Transportation:** Rome has an efficient public transportation system, including buses and metro. Consider purchasing a travel pass for convenient travel.
*   **Comfortable Shoes:** Wear comfortable shoes, as you'll be doing a lot of walking.

Enjoy your trip to Rome!


...

Okay, here is a detailed packing list for your 4-day relaxing trip to the Maldives, considering the water activities and resort relaxation.

#### Clothing
*   Swimsuits (2-3): Essential for daily swimming and water activities.
*   Swimsuit cover-up: For walking around the resort comfortably.
*   Lightweight, breathable dresses/skirts (2-3): Suitable for lunch, dinner, and relaxing on the beach.
*   Shorts (2): Comfortable for daytime activities and water sports.
*   T-shirts/Tank tops (3-4): Versatile for layering and casual wear.
*   Lightweight pants or sarong: For evenings or cooler moments.
*   Underwear: (5-6)
*   Pajamas/Sleepwear: For comfortable sleep.
*   Rash guard: To protect your skin during snorkeling, diving, and other water sports.
*   A light sweater or cardigan: Evenings can be breezy.

#### Footwear
*   Flip-flops or sandals: Essential for the beach and casual wear.
*   Water shoes: To protect your feet during water activities, especially snorkeling or diving.
*   Dressy sandals or wedges: For dinners at the resort.

#### Toiletries
*   Sunscreen: High SPF, water-resistant is crucial.
*   After-sun lotion: To soothe your skin after sun exposure.
*   Insect repellent: To protect against mosquitoes, especially in the evenings.
*   Shampoo, conditioner, and body wash: Some resorts provide these, but bring your favorites if you prefer.
*   Toothbrush, toothpaste, and floss
*   Deodorant
*   Makeup (if desired)
*   Hairbrush and hair ties
*   Face wash and moisturizer
*   Any personal hygiene items

#### Electronics
*   Phone and charger
*   Waterproof phone case: To protect your phone during water activities.
*   Camera or GoPro: To capture underwater moments and scenic views.
*   Chargers for all devices
*   Portable power bank: To recharge devices on the go.

#### Documents
*   Passport
*   Flight tickets
*   Hotel/Resort reservation confirmations
*   Copy of your passport (keep separate from your actual passport)
*   Travel insurance information
*   Driver's license (if you plan to rent any vehicles)
*   Scuba diving certification card (if applicable)

#### Other Essentials
*   Sunglasses: To protect your eyes from the strong sun.
*   Hat or visor: For sun protection.
*   Beach bag: To carry essentials to the beach or pool.
*   Reusable water bottle: To stay hydrated.
*   Snorkel gear (if you prefer your own): Although most resorts provide it.
*   Dry bag: To keep your valuables dry during boat trips and water activities.
*   Small backpack or daypack: For excursions and carrying essentials.
*   Books or e-reader: For relaxation.
*   Medications (if needed): Include any prescription and over-the-counter medications you may need.
*   Small first-aid kit: Band-aids, antiseptic wipes, pain relievers.
*   Cash and credit cards: Check if the resort accepts credit cards and if you need cash for smaller expenses or tips.

#### Outfit Suggestions for Key Activities
*   **Day 2 - Morning (Snorkeling Trip):** Swimsuit, rash guard (for sun protection), water shoes, waterproof sunscreen, and a hat. Bring your own snorkel gear if preferred, though most resorts provide it. Don't forget a waterproof camera to capture the underwater scenery!
*   **Day 2 - Evening (Sunset Dolphin Cruise/Beach Dinner):** A light, flowy dress or a nice top with lightweight pants. Accessorize with dressy sandals or wedges. A light sweater or cardigan may be useful as it can get breezy.
*   **Day 3 - Morning (Scuba Diving/Discover Scuba Diving):** Swimsuit and a rash guard to wear under the diving suit. If certified, bring your certification card. If doing a discovery dive, listen carefully to the instructor and don't worry! All gear will be provided.


--- Example 3: Workflow Agent Interaction Complete ---

--- Example 4: City Break & Shopping (Paris) ---
User Input: "I'm doing a 3-day city break in Paris in April. I want to visit famous landmarks, do some shopping, and enjoy cafes. Love Emily in Paris!"


Okay, here is a possible 3-day itinerary for your city break in Paris in April, keeping in mind your interest in famous landmarks, shopping, cafes, and *Emily in Paris*:

### Day 1

*   **Morning:** Start with a visit to the **Eiffel Tower**. Make sure to book your tickets in advance to avoid long queues.
*   **Lunch:** Have lunch at **Les Deux Magots**, one of the oldest and most famous cafes in Paris, known for its traditional French cuisine and historic ambiance.
*   **Afternoon:** Explore the **Louvre Museum**, home to iconic artworks like the Mona Lisa. Don't miss the **Louvre Pyramid**.
*   **Evening:** Dinner at **Terra Nera** (Gabriel's restaurant from *Emily in Paris*) in Place de l'Estrapade.

### Day 2

*   **Morning:** Take an "*Emily in Paris*" filming locations tour. This could include Emily's apartment, Gabriel's restaurant, and other recognizable spots from the show.
*   **Lunch:** Enjoy a pastry at **La Boulangerie Moderne**, the bakery where Emily tries her first pain au chocolat.
*   **Afternoon:** Shopping in **Le Marais**, known for its unique boutiques, concept stores, and designer shops.
*   **Evening:** Dinner in **Montmartre**, followed by a visit to the **Sacré-Cœur Basilica** for panoramic views of Paris.

### Day 3

*   **Morning:** Visit the **Arc de Triomphe**, and stroll down the **Champs-Élysées**, window shopping or indulging in some retail therapy.
*   **Lunch:** Have lunch at **Angelina**, famous for its hot chocolate and pastries.
*   **Afternoon:** Explore **Palais Royal Gardens** and see the **Colonnes de Buren**, a modern art installation featured in *Emily in Paris*.
*   **Evening:** Enjoy a scenic **Seine River cruise** to see the city's landmarks from a different perspective.

Enjoy your trip to Paris!


Okay, here is your packing list for a 3-day city break in Paris in April, tailored to your itinerary:

#### Clothing

*   **Comfortable walking shoes:** Essential for all the walking and exploring. Bring a pair that's already broken in.
*   **Stylish sneakers/fashionable walking shoes:** Paris is a stylish city, so consider shoes that are both comfortable and chic.
*   **Ankle boots or dressy flats:** Suitable for evenings out and nicer restaurants.
*   **Lightweight jacket or trench coat:** April in Paris can be unpredictable. A stylish, lightweight jacket or trench coat is perfect for layering.
*   **Sweaters or cardigans:** For layering on cooler days or evenings.
*   **Long-sleeved shirts/blouses:** Versatile for layering and suitable for both day and evening.
*   **T-shirts:** Basic layering pieces.
*   **Jeans or stylish trousers:** For everyday wear.
*   **Skirt or dress:** For dinners or a more dressed-up look.
*   **Socks:** Pack enough for each day, plus a couple of extra pairs.
*   **Underwear:** Enough for each day.
*   **Pajamas/sleepwear:** Comfortable sleepwear for the hotel.
*   **Scarf:** A stylish accessory that can also provide warmth.
*   **Gloves:** April can still be chilly, especially in the evenings.

#### Toiletries

*   **Toothbrush, toothpaste, and floss**
*   **Shampoo, conditioner, and body wash:** Consider travel-sized containers.
*   **Deodorant**
*   **Makeup:** As per your usual routine.
*   **Makeup remover**
*   **Sunscreen:** Even in April, sun protection is important.
*   **Moisturizer**
*   **Lip balm**
*   **Hand sanitizer**
*   **Wet wipes**
*   **Hairbrush and hair ties**
*   **Travel-sized tissues**
*   **Any personal hygiene items**
*   **Perfume/cologne**

#### Electronics

*   **Phone and charger**
*   **Portable power bank:** Useful for long days of sightseeing.
*   **European plug adapter:** If your devices use a different plug.
*   **Camera:** To capture all the beautiful moments.
*   **Headphones:** For listening to music or podcasts on the go.

#### Documents

*   **Passport**
*   **Visa:** If required for your nationality.
*   **Flight/train tickets**
*   **Hotel reservation confirmation**
*   **Copies of important documents:** Keep them separate from the originals.
*   **Travel insurance information**
*   **Emergency contact information**

#### Other Essentials

*   **Daypack:** For carrying essentials during the day.
*   **Reusable water bottle:** To stay hydrated.
*   **Small umbrella:** April showers are common in Paris.
*   **Sunglasses**
*   **Wallet with Euros and credit/debit cards**
*   **Phrasebook or translation app:** Although many people in Paris speak English, knowing some basic French phrases can be helpful.
*   **Snacks:** For quick energy boosts during sightseeing.
*   **Painkillers or any personal medication**
*   **Eye mask and earplugs:** For better sleep in the hotel.
*   **Small notebook and pen:** For jotting down notes and memories.

#### Outfit Suggestions for Key Activities

*   **Day 1 - Afternoon (Louvre Museum):** Smart casual. E.g., Comfortable trousers or dark jeans, a stylish top (blouse or nice t-shirt), a cardigan or light jacket, and comfortable walking shoes. A cross-body bag is ideal for security.
*   **Day 2 - Evening (Dinner in Montmartre):** Chic and comfortable. E.g., A dress with tights (if it's cool), ankle boots or dressy flats, a stylish jacket or coat. Accessorize with a scarf and jewelry.
*   **Day 3 - Afternoon (Palais Royal Gardens):** Trendy casual. E.g., Jeans or stylish pants, a fashionable top, a light jacket, and stylish sneakers or flats. Sunglasses and a crossbody bag are a must.


--- Example 4: Workflow Agent Interaction Complete ---

------ [all workflow agent interactions complete] ------


In [12]:
# Congratulations 🎉 
# You’ve just built and run a SequentialAgent workflow in ADK that first creates an itinerary,
# then generates a packing list tailored to your actual plans.
# Try changing the user input or expanding the workflow with more steps!